In [3]:
import torch
import torch.nn as nn 
import torchmetrics
from torch.utils.data import TensorDataset , DataLoader


In [4]:
#dataset

from sklearn.datasets import fetch_california_housing

In [5]:
#other libraries 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [6]:
housing = fetch_california_housing()

In [7]:
#lets scale

scale_data = StandardScaler()
housing_data = scale_data.fit_transform(housing.data)

scale_target = StandardScaler()
housing_target= scale_target.fit_transform(housing.target.reshape(-1 , 1)).flatten()

In [8]:
#data splittinh

x_train_full , x_test , y_train_full , y_test = train_test_split(housing_data , housing_target ,
                                                                  random_state=42 , test_size=0.15)

x_train , x_valid , y_train , y_valid = train_test_split(x_train_full , y_train_full ,
                                                         random_state=42 , test_size=0.15)

In [9]:
#convert to tensors 

x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
x_valid = torch.FloatTensor(x_valid)

y_train = torch.FloatTensor(y_train).reshape(-1  ,1 )
y_test = torch.FloatTensor(y_test).reshape(-1  ,1 )
y_valid = torch.FloatTensor(y_valid).reshape(-1  ,1 )



n_features = x_train.shape[1]

In [10]:
class WideAndDeep(nn.Module):
    def __init__(self , n_features):
        super().__init__()
        self.deep_stack = nn.Sequential(
            nn.Linear(n_features , 50),
            nn.ReLU(),
            nn.Linear(50 , 40),
            nn.ReLU(),
        )
        
        self.output_layer = nn.Linear(40 + n_features , 1)
        
    def forward(self , x):
        deep_output = self.deep_stack(x)
        wide_and_deep = torch.concat([x , deep_output] , dim=1)
        return self.output_layer(wide_and_deep)

In [11]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
def train(model , optimizer ,criterion ,train_loader , n_epoch):
    for epoch in range(n_epoch):
        total_loss = 0
        for x_batch , y_batch in train_loader:
            model.train()
            #on gpu
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            #forward
            y_pred = model(x_batch)
            
            #loss
            loss =  criterion(y_pred , y_batch)
            total_loss += loss.item()
            
            #backward
            loss.backward()
            
            #optimizer 
            
            optimizer.step()
            optimizer.zero_grad()
            
        mean_loss = total_loss/len(train_loader)
        
        print(f"Epoch : {epoch + 1}/{n_epoch}",
              f"Loss : {mean_loss:.4f}")
        
        
def evaluate(model , metrics , valid_loader):
    model.eval()
    metrics.reset()
    with torch.no_grad():
        for x_batch , y_batch in valid_loader:
            x_batch , y_batch = x_batch.to(device) , y_batch.to(device)
            y_pred = model(x_batch)
            metrics.update(y_pred , y_batch)
    
    return metrics.compute()


            

In [13]:
torch.manual_seed(42)
model = WideAndDeep(n_features).to(device)
learning_rate = 0.002
n_epoch = 20

train_dataset = TensorDataset(x_train , y_train)
train_loader = DataLoader(train_dataset , batch_size=100 , pin_memory=True , shuffle=True)

valid_dataset = TensorDataset(x_valid , y_valid)
valid_loader = DataLoader(valid_dataset , batch_size=100 , pin_memory=True , shuffle=True)

mse = torchmetrics.MeanSquaredError(squared=False).to(device)
loss = nn.MSELoss()

optimizer = torch.optim.SGD(params=model.parameters() , lr=learning_rate)

train(model , optimizer , loss , train_loader , n_epoch )
evaluate(model , mse , valid_loader)



Epoch : 1/20 Loss : 0.8552
Epoch : 2/20 Loss : 0.5839
Epoch : 3/20 Loss : 0.5115
Epoch : 4/20 Loss : 0.4899
Epoch : 5/20 Loss : 0.4714
Epoch : 6/20 Loss : 0.4616
Epoch : 7/20 Loss : 0.4533
Epoch : 8/20 Loss : 0.4458
Epoch : 9/20 Loss : 0.4404
Epoch : 10/20 Loss : 0.4340
Epoch : 11/20 Loss : 0.4285
Epoch : 12/20 Loss : 0.4252
Epoch : 13/20 Loss : 0.4213
Epoch : 14/20 Loss : 0.4153
Epoch : 15/20 Loss : 0.4136
Epoch : 16/20 Loss : 0.4094
Epoch : 17/20 Loss : 0.4058
Epoch : 18/20 Loss : 0.4025
Epoch : 19/20 Loss : 0.4024
Epoch : 20/20 Loss : 0.4003


tensor(0.6277, device='cuda:0')

In [14]:
def train_and_eval(model , optimizer , criterion  , metric , train_loader , 
                   valid_loader , n_epoch ):
    
    history = {"train_metrics" : [] , "val_metrics" : [] , "loss" : []}
    
    for epoch in range(n_epoch):
        
        metric.reset()
        total_loss = 0
        model.train()
        for x_batch , y_batch in train_loader:
            model.train()
            x_batch , y_batch = x_batch.to(device) , y_batch.to(device)
            
            #forward 
            y_pred = model(x_batch)
            
            #loss  
            loss = criterion(y_pred , y_batch)
            total_loss+= loss.item()
            
            #optimzer and backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            metric.update(y_pred , y_batch)
        
        mean_loss = total_loss/len(train_loader)
        history["loss"].append(mean_loss)
        history["train_metrics"].append(metric.compute().item())
        
        with torch.no_grad():
            model.eval()
            metric.reset()
            
            for x_valid_batch , y_valid_batch in valid_loader:
                x_valid_batch , y_valid_batch = x_valid_batch.to(device) , y_valid_batch.to(device)
                y_pred = model(x_valid_batch)
                metric.update(y_pred , y_valid_batch)
            
        history["val_metrics"].append(metric.compute().item())
        
        print(f"Epoch : {epoch+1}/{n_epoch}",
              f"Loss : {history['loss'][-1]:.4f}",
              f"Train Metrics : {history['train_metrics'][-1]:.4f}",
              f"valid metrics : {history['val_metrics'][-1]:.4f}")
        
    return history




torch.manual_seed(42)
model = WideAndDeep(n_features).to(device)
learning_rate = 0.002
n_epoch = 20

train_dataset = TensorDataset(x_train , y_train)
train_loader = DataLoader(train_dataset , batch_size=100 , pin_memory=True , shuffle=True)

valid_dataset = TensorDataset(x_valid , y_valid)
valid_loader = DataLoader(valid_dataset , batch_size=100 , pin_memory=True , shuffle=True)

rmse = torchmetrics.MeanSquaredError(squared=False).to(device)
loss = nn.MSELoss()

optimizer = torch.optim.SGD(params=model.parameters() , lr=learning_rate)

history = train_and_eval(model , optimizer , loss , rmse , train_loader , 
               valid_loader , n_epoch)


            
            

Epoch : 1/20 Loss : 0.8552 Train Metrics : 0.9266 valid metrics : 0.8128
Epoch : 2/20 Loss : 0.5837 Train Metrics : 0.7652 valid metrics : 0.7333
Epoch : 3/20 Loss : 0.5117 Train Metrics : 0.7161 valid metrics : 0.7053
Epoch : 4/20 Loss : 0.4852 Train Metrics : 0.6973 valid metrics : 0.6924
Epoch : 5/20 Loss : 0.4721 Train Metrics : 0.6873 valid metrics : 0.6836
Epoch : 6/20 Loss : 0.4614 Train Metrics : 0.6801 valid metrics : 0.6771
Epoch : 7/20 Loss : 0.4553 Train Metrics : 0.6740 valid metrics : 0.6710
Epoch : 8/20 Loss : 0.4477 Train Metrics : 0.6687 valid metrics : 0.6658
Epoch : 9/20 Loss : 0.4398 Train Metrics : 0.6638 valid metrics : 0.6608
Epoch : 10/20 Loss : 0.4355 Train Metrics : 0.6593 valid metrics : 0.6564
Epoch : 11/20 Loss : 0.4286 Train Metrics : 0.6554 valid metrics : 0.6524
Epoch : 12/20 Loss : 0.4239 Train Metrics : 0.6518 valid metrics : 0.6486
Epoch : 13/20 Loss : 0.4191 Train Metrics : 0.6485 valid metrics : 0.6453
Epoch : 14/20 Loss : 0.4162 Train Metrics : 0.6

In [15]:
class WideAndDeepV2(nn.Module):
    def __init__(self , n_features):
        super().__init__()
        self.deep_stack = nn.Sequential(
            nn.Linear(n_features - 2 , 50) , nn.ReLU(),
            nn.Linear(50 , 40) , nn.ReLU() , 
            nn.Linear(40 , 30)  , nn.ReLU()
        )
        
        self.output_layer=nn.Linear(30 + 5 , 1)
        
    def forward(self , x):
        x_wide = x[: , :5]
        x_deep = x[: , 2:]
        deep_output = self.deep_stack(x_deep)
        wide_and_deep = torch.concat([x_wide , deep_output] , dim=1)
        return self.output_layer(wide_and_deep)
    

torch.manual_seed(42)
model = WideAndDeepV2(n_features).to(device)
learning_rate = 0.002
n_epoch = 20

train_dataset = TensorDataset(x_train , y_train)
train_loader = DataLoader(train_dataset , batch_size=100 , pin_memory=True , shuffle=True)

valid_dataset = TensorDataset(x_valid , y_valid)
valid_loader = DataLoader(valid_dataset , batch_size=100 , pin_memory=True , shuffle=True)

rmse = torchmetrics.MeanSquaredError(squared=False).to(device)
loss = nn.MSELoss()

optimizer = torch.optim.SGD(params=model.parameters() , lr=learning_rate)

history = train_and_eval(model , optimizer , loss , rmse , train_loader , 
               valid_loader , n_epoch)



Epoch : 1/20 Loss : 0.9370 Train Metrics : 0.9677 valid metrics : 0.8529
Epoch : 2/20 Loss : 0.6365 Train Metrics : 0.7955 valid metrics : 0.7565
Epoch : 3/20 Loss : 0.5433 Train Metrics : 0.7356 valid metrics : 0.7223
Epoch : 4/20 Loss : 0.5099 Train Metrics : 0.7152 valid metrics : 0.7097
Epoch : 5/20 Loss : 0.5062 Train Metrics : 0.7072 valid metrics : 0.7033
Epoch : 6/20 Loss : 0.4990 Train Metrics : 0.7029 valid metrics : 0.6996
Epoch : 7/20 Loss : 0.4903 Train Metrics : 0.6998 valid metrics : 0.6965
Epoch : 8/20 Loss : 0.4853 Train Metrics : 0.6974 valid metrics : 0.6943
Epoch : 9/20 Loss : 0.4816 Train Metrics : 0.6952 valid metrics : 0.6922
Epoch : 10/20 Loss : 0.4791 Train Metrics : 0.6931 valid metrics : 0.6898
Epoch : 11/20 Loss : 0.4784 Train Metrics : 0.6912 valid metrics : 0.6877
Epoch : 12/20 Loss : 0.4751 Train Metrics : 0.6896 valid metrics : 0.6857
Epoch : 13/20 Loss : 0.4748 Train Metrics : 0.6877 valid metrics : 0.6840
Epoch : 14/20 Loss : 0.4690 Train Metrics : 0.6

In [16]:
#when input size is different 
#or wide and deep with multiple inputs 

In [25]:
class WideAndDeepV3(nn.Module):
    def __init__(self , n_features):
        super().__init__()
        self.deep_stack = nn.Sequential(
            nn.Linear(n_features - 2 , 50) , nn.ReLU() ,
            nn.Linear(50 , 40) , nn.ReLU(),
            nn.Linear(40 , 30) , nn.ReLU(),
        )
        self.output_layer = nn.Linear(30 + 5 , 1)
        
    def forward(self , x_wide , x_deep):
        deep_output = self.deep_stack(x_deep)
        wide_and_deep = torch.concat([x_wide , deep_output] , dim=1)
        return self.output_layer(wide_and_deep)

In [33]:
trian_data_wd = TensorDataset(x_train[: , :5] , x_train[: , 2:] , y_train)
train_loader_wd = DataLoader(trian_data_wd , shuffle=True , batch_size= 200 , pin_memory=True)

val_data_wd = TensorDataset(x_valid[: , :5] , x_valid[: , 2:] , y_valid)
val_loader_wd = DataLoader(val_data_wd , shuffle=True , batch_size= 200 , pin_memory=True)

test_data_wd = TensorDataset(x_test[: , :5] , x_test[: , 2:] , y_test)
test_loader_wd = DataLoader(test_data_wd , shuffle=True , pin_memory=True , batch_size= 200)




In [34]:
def train_and_eval_modified(model , optimizer , criterion  , metric , train_loader , 
                   valid_loader , n_epoch ):
    
    history = {"train_metrics" : [] , "val_metrics" : [] , "loss" : []}
    
    for epoch in range(n_epoch):
        
        metric.reset()
        total_loss = 0
        model.train()
        for x_batch_wide , x_batch_deep , y_batch in train_loader:
            model.train()
            x_batch_wide , x_batch_deep , y_batch = x_batch_wide.to(device) , x_batch_deep.to(device) ,  y_batch.to(device)
            
            #forward 
            y_pred = model(x_batch_wide , x_batch_deep)
            
            #loss  
            loss = criterion(y_pred , y_batch)
            total_loss+= loss.item()
            
            #optimzer and backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            metric.update(y_pred , y_batch)
        
        mean_loss = total_loss/len(train_loader)
        history["loss"].append(mean_loss)
        history["train_metrics"].append(metric.compute().item())
        
        with torch.no_grad():
            model.eval()
            metric.reset()
            
            for x_valid_batch_wide , x_valid_batch_deep , y_valid_batch in valid_loader:
                x_valid_batch_wide , x_valid_batch_deep , y_valid_batch = x_valid_batch_wide.to(device) , x_valid_batch_deep.to(device) , y_valid_batch.to(device)
                y_pred = model(x_valid_batch_wide , x_valid_batch_deep)
                metric.update(y_pred , y_valid_batch)
            
        history["val_metrics"].append(metric.compute().item())
        
        print(f"Epoch : {epoch+1}/{n_epoch}",
              f"Loss : {history['loss'][-1]:.4f}",
              f"Train Metrics : {history['train_metrics'][-1]:.4f}",
              f"valid metrics : {history['val_metrics'][-1]:.4f}")
        
    return history




torch.manual_seed(42)
model = WideAndDeepV3(n_features).to(device)
learning_rate = 0.002
n_epoch = 20

rmse = torchmetrics.MeanSquaredError(squared=False).to(device)
loss = nn.MSELoss()

optimizer = torch.optim.SGD(params=model.parameters() , lr=learning_rate)

history = train_and_eval_modified(model , optimizer , loss , rmse , train_loader_wd , 
               val_loader_wd , n_epoch)


            
            

Epoch : 1/20 Loss : 1.0636 Train Metrics : 1.0315 valid metrics : 0.9539
Epoch : 2/20 Loss : 0.8074 Train Metrics : 0.8985 valid metrics : 0.8530
Epoch : 3/20 Loss : 0.6698 Train Metrics : 0.8180 valid metrics : 0.7920
Epoch : 4/20 Loss : 0.5934 Train Metrics : 0.7706 valid metrics : 0.7561
Epoch : 5/20 Loss : 0.5541 Train Metrics : 0.7432 valid metrics : 0.7350
Epoch : 6/20 Loss : 0.5296 Train Metrics : 0.7273 valid metrics : 0.7224
Epoch : 7/20 Loss : 0.5163 Train Metrics : 0.7180 valid metrics : 0.7146
Epoch : 8/20 Loss : 0.5065 Train Metrics : 0.7122 valid metrics : 0.7096
Epoch : 9/20 Loss : 0.5024 Train Metrics : 0.7084 valid metrics : 0.7062
Epoch : 10/20 Loss : 0.4987 Train Metrics : 0.7057 valid metrics : 0.7035
Epoch : 11/20 Loss : 0.4948 Train Metrics : 0.7036 valid metrics : 0.7014
Epoch : 12/20 Loss : 0.4926 Train Metrics : 0.7020 valid metrics : 0.6996
Epoch : 13/20 Loss : 0.4912 Train Metrics : 0.7005 valid metrics : 0.6981
Epoch : 14/20 Loss : 0.4879 Train Metrics : 0.6

In [35]:
#naming each input 

class WideAndDeepDataset(torch.utils.data.Dataset):
    def __init__(self , x_wide , x_deep , y):
        self.x_wide = x_wide
        self.x_deep = x_deep
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self , idx):
        input_dict = {"x_wide" : self.x_wide[idx] , "x_deep" : self.x_deep[idx]}
        return input_dict , self.y[idx]
    
    
train_data_named = WideAndDeepDataset(
    x_wide = x_train[: , :5] , x_deep = x_train[: , 2:] , y=y_train
)

train_loader_named = DataLoader(train_data_named , batch_size = 200 , shuffle=True)
    
    